## LibRecommender models experiments

* https://github.com/massquantity/LibRecommender/tree/master/examples

* Try different models. 
* Note that we explicitly create negatives.
* Can do train test split by time or other things


* Data Format
    * One thing is important, the model assumes that user, item, and label column index are 0, 1, and 2, respectively. You may wish to change the column order if that's not the case.

Possible approahc + negatives - https://github.com/zhangruiskyline/DeepLearning/blob/master/doc/Recommendation.md#ranking 


* Negative sampling from the sparse user-item cooccurrence matrix
    * https://stackoverflow.com/questions/49971318/how-to-generate-negative-samples-in-tensorflow


In [1]:
# from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
import numpy as np
import random
np.random.seed(100)
random.seed(100)
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, GroupShuffleSplit

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import TopKCategoricalAccuracy, Precision, SparseTopKCategoricalAccuracy # @4
from tensorflow.keras.utils import to_categorical
from tensorflow import feature_column
from tensorflow.keras import layers

%matplotlib inline

pd.set_option("display.max_columns", 90)

In [2]:
from libreco.data import split_by_ratio_chrono, DatasetFeat, split_by_ratio
from libreco.algorithms import (
    FM, WideDeep, DeepFM, AutoInt, DIN, YouTubeMatch, YouTubeRanking
)


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
def reset_state(name):
    tf.compat.v1.reset_default_graph()
    print("\n", "=" * 30, name, "=" * 30)

In [4]:
!nvidia-smi -L

GPU 0: GeForce RTX 2060 (UUID: GPU-d8cefda9-d4cb-990c-cc01-a2a4f2416484)


In [6]:
tf.compat.v1.keras.layers.enable_v2_dtype_behavior()
## https://www.tensorflow.org/guide/mixed_precision ## TF mixed precision - pytorch requires other setup
from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
## will need to correct in places, e.g.: 
## outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)

#### Features to add:
* Lag 
* Rank (popularity) of city, country (in general, +- given booker country)
* Count of hotel; user, trip size ? (may be leaky )
* Seasonal features - Holidays? , datetime

Aggregate feats:
* user changed country? last booking (lag 1) country change? 
* max/min/avg popularity rank of previous locations visited



We should create a dictionary of the rank, count, city/country etc' feats, so we can easily merge them when making more "negative" samples/feats for ranking.


* Consider using a df2 of df without dates + drop_duplicates, +- without user/trip id (After calcing that) .


Leaky or potentially leaky (Dependso n test set): 
* Target freq features - frequency of target city, given source county +- affiliate +- month of year +- given country (and interactions of target freq). 
    * Risk of leaks - depends of test data has temporal split or not. 
    * cartboost can do target encode, but this lets us do it for interactions, e.g. target city freq given the 2 countries and affiliate.
    * beware overfitting! 

In [7]:
MIN_TARGET_FREQ = 25 # drop target/city_id values that appear less than this many times, as final step's target 
KEEP_TOP_K_TARGETS = 0 # keep K most frequent city ID targets (redundnat with the above, )

## (some) categorical variables that appear less than this many times will be replaced with a placeholder value!
## Includes CITY id (but done after target filtering, to avoid creating a "rare class" target:
LOW_COUNT_THRESH = 10

RUN_TABNET = False
max_epochs = 20

GET_COUNT_AGG_FEATS = False ## disable getting count, rank etc' groupby features , for speedup

DROP_FIRST_ROW =  True #False  ## drop first interaction per user from train data

## for matrix factorization/CF:
### morte possible ID_cols :  # last (last step in trip) - would double data per user incorrectly
### hotel_country_lag1 , city_id_lag1  (very relevant - needs shared embeddingm and would increase cardinality a lot.. ) 
ID_COLS = [
#     'device_class',
#            'affiliate_id',
           'booker_country',
           'checkin_quarter',
#            "last",
          "first_hotel_country"] 
MF_KEEP_COLS = ["ID"]+ID_COLS+['city_id',"hotel_country"]

SAVE_TO_DISK = False

TARGET_COL =  'city_id' #"city_id"#'hotel_country' 
USER_ID_COL = "utrip_id"

In [8]:
# most basic categorical columns , without 'user_id', , 'utrip_id' ordevice_class - used for count encoding/filtering
BASE_CAT_COLS = ['city_id',  'affiliate_id', 'booker_country', 'hotel_country']

# ### features to get lags for. Not very robust. May want different feats for lags before -1
# LAG_FEAT_COLS = ['city_id', 'device_class',
#        'affiliate_id', 'booker_country', 'hotel_country', 
#        'duration', 'same_country', 'checkin_day', 'checkin_weekday',
#        'checkin_week',
#         'checkout_weekday','checkout_week',
#        'city_id_count', 'affiliate_id_count',
#        'booker_country_count', 'hotel_country_count', 
#        'checkin_month_count', 'checkin_week_count', 'city_id_nunique',
#        'affiliate_id_nunique', 'booker_country_nunique',
#        'hotel_country_nunique', 'city_id_rank_by_hotel_country',
#        'city_id_rank_by_booker_country', 'city_id_rank_by_affiliate',
#        'affiliate_id_rank_by_hotel_country',
#        'affiliate_id_rank_by_booker_country', 'affiliate_id_rank_by_affiliate',
#        'booker_country_rank_by_hotel_country',
#        'booker_country_rank_by_booker_country',
#        'booker_country_rank_by_affiliate',
#        'hotel_country_rank_by_hotel_country',
#        'hotel_country_rank_by_booker_country',
#        'hotel_country_rank_by_affiliate',
#        'checkin_month_rank_by_hotel_country',
#        'checkin_month_rank_by_booker_country',
#        'checkin_month_rank_by_affiliate']

In [9]:
# https://stackoverflow.com/questions/33907537/groupby-and-lag-all-columns-of-a-dataframe
# https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns
## lag features with groupby over many columns: 
def groupbyLagFeatures(df:pd.DataFrame,lag:[]=[1,2],group="utrip_id",lag_feature_cols=[]):
    """
    lag features with groupby over many columns.
    Assumes sorted data!
    https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns"""
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    else:
         df=pd.concat([df]+[df.groupby(group).shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    return df


def groupbyFirstLagFeatures(df:pd.DataFrame,group="user_id",lag_feature_cols=[]):
    """
    Get  first/head value lag-like of features with groupby over columns. Assumes sorted data!
    """
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].transform("first").add_prefix("first_")],axis=1)
    else:
#          df=pd.concat([df]+[df.groupby(group).first().add_prefix("first_")],axis=1)
        df=pd.concat([df]+[df.groupby(group).transform("first").add_prefix("first_")],axis=1)
    return df

######## Get n most popular items, per group
def most_popular(group, n_max=4):
    """Find most popular hotel clusters by destination
    Define a function to get most popular hotels for a destination group.

    Previous version used nlargest() Series method to get indices of largest elements. But the method is rather slow.
    Source: https://www.kaggle.com/dvasyukova/predict-hotel-type-with-pandas
    """
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets


# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=64,target_col="target"):
    """
    Wrap dataframes with tf.data. 
    This will enable us to use feature columns as a bridge to map from the columns in a dataframe to features used to train the model.
    https://www.tensorflow.org/tutorials/structured_data/feature_columns#create_an_input_pipeline_using_tfdata
    """
    dataframe = dataframe.copy()
    labels = dataframe.pop(target_col)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds


import multiprocessing
import numpy as np
import pandas as pd


In [10]:
df = pd.read_csv("booking_train_set.csv",
#                  nrows=323456,
                 index_col=[0],
                 parse_dates=["checkin"],infer_datetime_format=True)

df.sort_values([ "checkin",
                "user_id"],inplace=True)
print(df.nunique())
df

user_id           200153
checkin              425
checkout             425
city_id            39901
device_class           3
affiliate_id        3254
booker_country         5
hotel_country        195
utrip_id          217686
dtype: int64


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
519017,727105,2015-12-31,2016-01-01,18820,mobile,359,The Devilfire Empire,Cobra Island,727105_1
986596,2000964,2015-12-31,2016-01-01,63341,mobile,8151,The Devilfire Empire,Cobra Island,2000964_1
7504,2595109,2015-12-31,2016-01-01,27404,mobile,359,The Devilfire Empire,Cobra Island,2595109_1
52884,110418,2016-01-01,2016-01-02,3763,desktop,9924,The Devilfire Empire,Glubbdubdrib,110418_1
1068508,221863,2016-01-01,2016-01-02,45597,mobile,7774,Gondal,Gondal,221863_1
...,...,...,...,...,...,...,...,...,...
908844,5755992,2017-02-27,2017-02-28,52860,desktop,4568,Gondal,Axphain,5755992_4
1154024,5842454,2017-02-27,2017-02-28,35850,desktop,384,Gondal,Rook Islands,5842454_6
1160065,5936647,2017-02-27,2017-02-28,20199,desktop,4541,Gondal,Kasnia,5936647_2
244942,5955565,2017-02-27,2017-02-28,54384,desktop,8132,Elbonia,Patusan,5955565_2


In [11]:
### i disabled most of thefeature eztraction here for simplicity

# df["duration"] = (df["checkout"] - df["checkin"]).dt.days
# df["same_country"] = (df["booker_country"]==df["hotel_country"]).astype(int)

# df["checkin_day"] = df["checkin"].dt.day
# df["checkin_weekday"] = df["checkin"].dt.weekday
df["checkin_week"] = df["checkin"].dt.isocalendar().week.astype(int) ## week of year
df["checkin_month"] = df["checkin"].dt.month
# df["checkin_year"] = df["checkin"].dt.year-2016

df["checkin_quarter"] = df["checkin"].dt.quarter # relatively redundant but may be used for "id"

# df["checkin_quarter"] = df["checkin_quarter"]/4 # scale. could also do cos, sin extraction. makesi t a float instead of int/embedding


# df["checkout_weekday"] = df["checkout"].dt.weekday
# df["checkout_week"] = df["checkout"].dt.isocalendar().week.astype(int) ## week of year
# df["checkout_day"] = df["checkout"].dt.day ## day of month

## cyclical datetime embeddings
## drop originakl variables? 
## TODO:L add for other variables, +- those that we'll embed (week?)

# df['checkin_weekday_sin'] = np.sin(df["checkin_weekday"]*(2.*np.pi/7))
# df['checkin_weekday_cos'] = np.cos(df["checkin_weekday"]*(2.*np.pi/7))
# df['checkin_month_sin'] = np.sin((df["checkin_month"]-1)*(2.*np.pi/12))
# df['checkin_month_cos'] = np.cos((df["checkin_month"]-1)*(2.*np.pi/12))

# #############
# # last number in utrip id - probably which trip number it is:
# df["utrip_number"] = df["utrip_id"].str.split("_",expand=True)[1].astype(int)

### encode string columns - must be consistent with test data 
### IF we can concat test with train, we can just do a single transformation  for the NON TARGET cols
# obj_cols_list = df.select_dtypes("O").columns.values
obj_cols_list = ['device_class','booker_country','hotel_country',
#                 "city_id"
                ] # we could also define when loading data, dtype

for c in obj_cols_list:
    df[c] = df[c].astype("category")
#     df[c] = df[c].cat.codes.astype(int)
#     print("min",df[c].min()) min is 0 - which is what the embedding (indices) will expect

## view steps of a trip per user & trip, in order. ## last step == 1.
## count #/pct step in a trip (utrip_id) per user. Useful to get the "final" step per trip - for prediction
## note that the order is ascending, so we would need to select by "last" . (i.e "1" is the first step, 2 the second, etc') , or we could use pct .rank(ascending=True,pct=True)
#### this feature overlaps with the count of each trip id (for the final row)
##  = df.sort_values(["checkin","checkout"])... - df already sorted above
# df["utrip_steps_from_end"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=True) 

In [12]:
### add features to be consistent with test set of row in trip, and total trips in trip

df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)

### last step in trip
df["last"] = (df["total_rows"]==df["row_num"]).astype(int)

df["total_rows"].describe();

### replace rare categorical variable(s) - affiliates
* replace rare variables (under 2 occurrences) with "-1" dummy

In [13]:
### replace rare variables (under 2 occurrences) with "-1" dummy

affiliates_counts = df["affiliate_id"].value_counts()
print("before:", affiliates_counts)
print("uniques",df["affiliate_id"].nunique())
affiliates_counts = affiliates_counts.to_dict()
# df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].apply(lambda x: x.map(x.value_counts()))>=3, -1)
df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].map(affiliates_counts)>=3, -2)
df["affiliate_id"] = df["affiliate_id"].astype(int)

print("after\n",df["affiliate_id"].value_counts())
print("uniques",df["affiliate_id"].nunique())

before: 9924     277775
359      171385
384       88137
9452      85476
4541      41504
          ...  
8351          1
8464          1
2202          1
10513         1
2047          1
Name: affiliate_id, Length: 3254, dtype: int64
uniques 3254
after
 9924    277775
359     171385
384      88137
9452     85476
4541     41504
         ...  
2615         3
5963         3
2618         3
838          3
176          3
Name: affiliate_id, Length: 2152, dtype: int64
uniques 2152


In [14]:
# df["total_rows"].map(pd.cut(df["total_rows"],bins=3,))

#### Add LAG feature(s)  + DROP First visited location rows
* "first" hotel country (vs most recent country visited)
    * `groupbyFirstLagFeatures(df,group="user_id",lag_feature_cols=["hotel_country","city_id"])`
* Can consider: lag1 hotel_country, hotel_id 

In [15]:
## add the "first" place visited/values
### nopte - will need to drop first row in trip, or impute nans when using this feature 

### first by user results in too much sparsity/rareness for our IDs purposes
# df = groupbyFirstLagFeatures(df,group="user_id",lag_feature_cols=["hotel_country"]) # ["hotel_country","city_id"]

## alt - messy, but maybe good enough : 
df = groupbyFirstLagFeatures(df,group=['device_class', 'affiliate_id',
                                       'booker_country','checkin_month',"last"],
                             lag_feature_cols=["hotel_country","city_id",'device_class', 'affiliate_id',
                                              "checkin_quarter","checkin_month","booker_country"])


print(df[["first_hotel_country","hotel_country","city_id","first_city_id"]].nunique())
df

first_hotel_country      144
hotel_country            195
city_id                39901
first_city_id           7863
dtype: int64


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,checkin_week,checkin_month,checkin_quarter,row_num,total_rows,last,first_hotel_country,first_city_id,first_device_class,first_affiliate_id,first_checkin_quarter,first_checkin_month,first_booker_country
519017,727105,2015-12-31,2016-01-01,18820,mobile,359,The Devilfire Empire,Cobra Island,727105_1,53,12,4,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire
986596,2000964,2015-12-31,2016-01-01,63341,mobile,8151,The Devilfire Empire,Cobra Island,2000964_1,53,12,4,1,5,0,Cobra Island,63341,mobile,8151,4,12,The Devilfire Empire
7504,2595109,2015-12-31,2016-01-01,27404,mobile,359,The Devilfire Empire,Cobra Island,2595109_1,53,12,4,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire
52884,110418,2016-01-01,2016-01-02,3763,desktop,9924,The Devilfire Empire,Glubbdubdrib,110418_1,53,1,1,1,10,0,Glubbdubdrib,3763,desktop,9924,1,1,The Devilfire Empire
1068508,221863,2016-01-01,2016-01-02,45597,mobile,7774,Gondal,Gondal,221863_1,53,1,1,1,8,0,Gondal,45597,mobile,7774,1,1,Gondal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908844,5755992,2017-02-27,2017-02-28,52860,desktop,4568,Gondal,Axphain,5755992_4,9,2,1,5,5,1,Gondal,29394,desktop,4568,1,2,Gondal
1154024,5842454,2017-02-27,2017-02-28,35850,desktop,384,Gondal,Rook Islands,5842454_6,9,2,1,9,9,1,Sylvania,14342,desktop,384,1,2,Gondal
1160065,5936647,2017-02-27,2017-02-28,20199,desktop,4541,Gondal,Kasnia,5936647_2,9,2,1,12,12,1,Rolisica,8462,desktop,4541,1,2,Gondal
244942,5955565,2017-02-27,2017-02-28,54384,desktop,8132,Elbonia,Patusan,5955565_2,9,2,1,5,5,1,Elbonia,52327,desktop,8132,1,2,Elbonia


In [16]:
df.nunique()

user_id                  200153
checkin                     425
checkout                    425
city_id                   39901
device_class                  3
affiliate_id               2152
booker_country                5
hotel_country               195
utrip_id                 217686
checkin_week                 53
checkin_month                12
checkin_quarter               4
row_num                      48
total_rows                   41
last                          2
first_hotel_country         144
first_city_id              7863
first_device_class            3
first_affiliate_id         2152
first_checkin_quarter         4
first_checkin_month          12
first_booker_country          5
dtype: int64

* Continue

In [17]:
##### Following aggregation features - would be best to use time window (sort data) to generate, otherwise they will LEAK! (e.g. nunique countries visited)

### count features (can also later add rank inside groups).
### Some may be leaks (# visits in a trip should use time window?) , and do users repeat? 
### can add more counts of group X time period (e.g. affiliate X month of year)

## alt way to get counts/freq :

if GET_COUNT_AGG_FEATS:
    count_cols = [ 'city_id','affiliate_id', 'booker_country', 'hotel_country', 
    #               'utrip_id','user_id', 
     "checkin_month","checkin_week"]
    for c in count_cols:
        df[f"{c}_count"] = df.groupby([c])["duration"].transform("size")

    ########################################################
    ## nunique per trip
    ### https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series

    nunique_cols = [ 'city_id','affiliate_id', 'booker_country', 'hotel_country']
    # df["nunique_booker_countries"] = df.groupby("utrip_id")["booker_country"].nunique()
    # df["nunique_hotel_country"] = df.groupby("utrip_id")["hotel_country"].nunique()
    for c in nunique_cols:
        df[f"{c}_nunique"] = df.groupby(["utrip_id"])[c].transform("nunique")
    print(df.nunique())

    ########################################################
    ## get frequency/count feature's rank within a group - e.g. within a country (or affiliate) 
    ## add "_count" to column name to get count col name, then add rank col 

    ### ALT/ duplicate feat - add percent rank (instead or in addition)

    rank_cols = ['city_id','affiliate_id', 'booker_country','hotel_country',
     "checkin_month"]
    ### what is meaning of groupby and rank of smae variable by same var? Surely should be 1 / unary? 
    for c in rank_cols:
        df[f"{c}_rank_by_hotel_country"] = df.groupby(['hotel_country'])[f"{c}_count"].transform("rank")
        df[f"{c}_rank_by_booker_country"] = df.groupby(['booker_country'])[f"{c}_count"].transform("rank")
        df[f"{c}_rank_by_affiliate"] = df.groupby(['affiliate_id'])[f"{c}_count"].transform("rank")     
else:
    freq = df["city_id"].value_counts()
    df["city_id_count"] = df["city_id"].map(freq)
    print(freq.describe())
    
df

count    39901.000000
mean        29.243252
std        218.801654
min          1.000000
25%          1.000000
50%          3.000000
75%          9.000000
max      11242.000000
Name: city_id, dtype: float64


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,checkin_week,checkin_month,checkin_quarter,row_num,total_rows,last,first_hotel_country,first_city_id,first_device_class,first_affiliate_id,first_checkin_quarter,first_checkin_month,first_booker_country,city_id_count
519017,727105,2015-12-31,2016-01-01,18820,mobile,359,The Devilfire Empire,Cobra Island,727105_1,53,12,4,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire,535
986596,2000964,2015-12-31,2016-01-01,63341,mobile,8151,The Devilfire Empire,Cobra Island,2000964_1,53,12,4,1,5,0,Cobra Island,63341,mobile,8151,4,12,The Devilfire Empire,196
7504,2595109,2015-12-31,2016-01-01,27404,mobile,359,The Devilfire Empire,Cobra Island,2595109_1,53,12,4,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire,3614
52884,110418,2016-01-01,2016-01-02,3763,desktop,9924,The Devilfire Empire,Glubbdubdrib,110418_1,53,1,1,1,10,0,Glubbdubdrib,3763,desktop,9924,1,1,The Devilfire Empire,5544
1068508,221863,2016-01-01,2016-01-02,45597,mobile,7774,Gondal,Gondal,221863_1,53,1,1,1,8,0,Gondal,45597,mobile,7774,1,1,Gondal,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908844,5755992,2017-02-27,2017-02-28,52860,desktop,4568,Gondal,Axphain,5755992_4,9,2,1,5,5,1,Gondal,29394,desktop,4568,1,2,Gondal,12
1154024,5842454,2017-02-27,2017-02-28,35850,desktop,384,Gondal,Rook Islands,5842454_6,9,2,1,9,9,1,Sylvania,14342,desktop,384,1,2,Gondal,692
1160065,5936647,2017-02-27,2017-02-28,20199,desktop,4541,Gondal,Kasnia,5936647_2,9,2,1,12,12,1,Rolisica,8462,desktop,4541,1,2,Gondal,12
244942,5955565,2017-02-27,2017-02-28,54384,desktop,8132,Elbonia,Patusan,5955565_2,9,2,1,5,5,1,Elbonia,52327,desktop,8132,1,2,Elbonia,526


In [18]:
print("cities with more than 7 occurences:",df.loc[df["city_id_count"]>=7]["city_id"].nunique())
print(f"cities with more than MIN_TARGET_FREQ ({MIN_TARGET_FREQ}) occurences:",df.loc[df["city_id_count"]>=MIN_TARGET_FREQ]["city_id"].nunique())
print(f"rows left if filtering by MIN_TARGET_FREQ :",df.loc[df["city_id_count"]>=MIN_TARGET_FREQ].shape[0])

cities with more than 7 occurences: 12345
cities with more than MIN_TARGET_FREQ (25) occurences: 4822
rows left if filtering by MIN_TARGET_FREQ : 1010884


In [19]:
assert df.isna().sum().max() ==0

In [20]:
# df2 = df[["user_id","city_id"]].drop_duplicates().copy()
df2 = df.drop_duplicates(subset=["user_id","city_id"])["city_id"].copy()
print(df2.shape[0])
print("df2 nunique (cities without duplicate user visits)",df2.nunique())

# c2_counts = df2["city_id"].value_counts()
c2_counts = df2.value_counts()
# df2["new_counts"] = df2["city_id"].map(c2_counts)
# df2["new_counts"] = df2.map(c2_counts)
print("city counts")
print(c2_counts)
print(c2_counts.describe())
print("cities with at least 2:",(c2_counts>=2).sum())
print("cities with at least 3:",(c2_counts>=3).sum())
print("cities with at least 5:",(c2_counts>=5).sum())
print("cities with at least 10:",(c2_counts>=10).sum())
print("cities with at least 15:",(c2_counts>=15).sum())
print("cities with at least 20:",(c2_counts>=20).sum())
print("cities with at least 30:",(c2_counts>=30).sum())
print("cities with at least 50:",(c2_counts>=50).sum())
print("cities with at least 100:",(c2_counts>=100).sum())

c2_freq = df2.value_counts(normalize=True)
print("top 4 sum coverage (normalized): ",c2_freq[0:4].sum().round(3))
print("top 50 sum coverage (normalized): ",c2_freq[0:50].sum().round(3))
print("top 100 sum coverage (normalized): ",c2_freq[0:100].sum().round(3))
print("top 400 sum coverage (normalized): ",c2_freq[0:400].sum().round(3))
print("top 1,000 sum coverage (normalized): ",c2_freq[0:1000].sum().round(3))
print("top 5,000 sum coverage (normalized): ",c2_freq[0:5000].sum().round(3))
print("top 8,000 sum coverage (normalized): ",c2_freq[0:8000].sum().round(3))
print("top 15,000 sum coverage (normalized): ",c2_freq[0:15000].sum().round(3))

1029804
df2 nunique (cities without duplicate user visits) 39901
city counts
23921    8137
55128    7197
47499    7188
64876    6724
29319    6361
         ... 
50916       1
57063       1
46826       1
38638       1
2049        1
Name: city_id, Length: 39901, dtype: int64
count    39901.000000
mean        25.808977
std        173.750203
min          1.000000
25%          1.000000
50%          3.000000
75%          9.000000
max       8137.000000
Name: city_id, dtype: float64
cities with at least 2: 27533
cities with at least 3: 21337
cities with at least 5: 15248
cities with at least 10: 9281
cities with at least 15: 6836
cities with at least 20: 5499
cities with at least 30: 4006
cities with at least 50: 2673
cities with at least 100: 1550
top 4 sum coverage (normalized):  0.028
top 50 sum coverage (normalized):  0.183
top 100 sum coverage (normalized):  0.263
top 400 sum coverage (normalized):  0.481
top 1,000 sum coverage (normalized):  0.64
top 5,000 sum coverage (normalized):  0.8

## Frequent city target List + City count encoding
* Get the K most frequent target city IDs - selected based on frequency as final destination (not just overall)
* +- Also after this, replace rare city IDs categorical features with count encoding to reduce dimensionality
    * Keep them as count, or aggregate all of them as "under_K"?

##### Output  : `TOP_TARGETS` - filter data by this *after* creation of lag features ! 

* Drop duplicates by the same user (reduce possible bias of frequent users? Only relevant if test is seperater from "frequent travellers") 
    * results in 216,633 , vs 217,686 without dropping duplicates by users
    * ~19.9k unique cities
    
* Could do other encodings - https://contrib.scikit-learn.org/category_encoders/count.html

* Note that all this is after we've added rank, count features beforehand, so that information won't be lost for these variables, despite these transforms



* **NOTE** he most frequent final destinations are NOT the same as the most popular overall destinations +- first location ! 

In [21]:
if KEEP_TOP_K_TARGETS > 0 :
    df_end = df.loc[df["utrip_steps_from_end"]==1].drop_duplicates(subset=["city_id","hotel_country","user_id"])[["city_id","hotel_country"]].copy()
    print(df_end.shape[0])
    end_city_counts = df_end.city_id.value_counts()
    print(end_city_counts)
    
    TOP_TARGETS = end_city_counts.head(KEEP_TOP_K_TARGETS).index.values
    print(f"top {KEEP_TOP_K_TARGETS} targets \n",TOP_TARGETS)
    
#     assert df.loc[df["city_id"].isin(TOP_TARGETS)]["city_id"].nunique() == KEEP_TOP_K_TARGETS

####        
# replace low frequency categoircal features    

# ##replace with count encoding if have at least k, group rarest as "-1":# df[BASE_CAT_COLS] = df[BASE_CAT_COLS].where(df[BASE_CAT_COLS].apply(lambda x: x.map(x.value_counts()))>=LOW_COUNT_THRESH, -1)   
# ## replace/group only the rare variables : 
# df[BASE_CAT_COLS] = df[BASE_CAT_COLS].where(df[BASE_CAT_COLS].apply(lambda x: x.map(x.value_counts()))>=LOW_COUNT_THRESH, -1)
# df[BASE_CAT_COLS].head()

### Filter data by city id frequency
#### df2 - smaller df (may not ben ecessary to make

* drop rows if it's city id appears less than X times - this is prior to CF
* We could also add inclusion/exclusion based on target appearing/frequency as target in final stage of rtip - optional
* maybe also drop (end exclude in freq counting) thefirst point in a trip ? 



In [22]:
### unsure about this filtering - depends if data points are real or mistake
print("dropping users with less than 4 trips")
df = df.loc[df["total_rows"]>=4]#.copy()
# print("abnormal users dropped",df.shape[0]-df.shape[0])

print(f"dropping cities with less than {MIN_TARGET_FREQ} occurences:")
df = df.loc[df.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ] ## update count
# df = df.loc[df["city_id_count"]>=MIN_TARGET_FREQ]
print(df.shape[0])

# print(f"dropping users with less than 4 instances, after previous city filter:")
df = df.loc[df.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]

# print(f"dropping cities with less than {MIN_TARGET_FREQ} occurences:")
df = df.loc[df.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]

print(f"dropping users with less than 4 instances, after previous filters:")
df = df.loc[df.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]

df = df.loc[df.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]
df = df.loc[df.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]
df = df.loc[df.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]
df = df.loc[df.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]
# df = df.loc[df.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]
# df = df.loc[df.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]

print("rows left:",df.shape[0])

print("nunique cities after freq filt",df[["city_id","utrip_id","user_id","hotel_country"]].nunique())
print("nunique city_id per hotel_country:")
df.groupby(["hotel_country"])["city_id"].nunique().describe()

dropping users with less than 4 trips
dropping cities with less than 25 occurences:
1008673
dropping users with less than 4 instances, after previous filters:
rows left: 857829
nunique cities after freq filt city_id            3667
utrip_id         162904
user_id          152608
hotel_country       105
dtype: int64
nunique city_id per hotel_country:


count    195.000000
mean      18.805128
std       54.053457
min        0.000000
25%        0.000000
50%        1.000000
75%       11.500000
max      429.000000
Name: city_id, dtype: float64

* drop the first location visited per trip, as that shares the "first country/hotel/city id feature" ??

In [23]:
if DROP_FIRST_ROW: ## drop the first location visited per trip, (row_num = 1) from df data/interactions
    df = df.loc[df["row_num"]>1]
    print(df.shape[0])

702158


In [24]:
df.sort_values([ "checkin",
                "utrip_id"],inplace=True)

In [25]:
df["label"] = 1

## rearrange columns - user, item, and label column index are 0, 1, and 2
# new_columns = cols_to_order + (frame.columns.drop(cols_to_order).tolist())
cols_to_order = ["utrip_id","city_id","label","checkin"]
new_columns = cols_to_order + (df.columns.drop(cols_to_order).tolist())

df = df[new_columns]
df

,utrip_id,city_id,label,checkin,user_id,checkout,device_class,affiliate_id,booker_country,hotel_country,checkin_week,checkin_month,checkin_quarter,row_num,total_rows,last,first_hotel_country,first_city_id,first_device_class,first_affiliate_id,first_checkin_quarter,first_checkin_month,first_booker_country,city_id_count
519018,727105_1,60403,1,2016-01-01,727105,2016-01-02,mobile,359,The Devilfire Empire,Cobra Island,53,1,1,2,4,0,Cobra Island,60403,mobile,359,1,1,The Devilfire Empire,565
805583,1032571_1,48483,1,2016-01-02,1032571,2016-01-04,mobile,9924,The Devilfire Empire,Cobra Island,53,1,1,2,5,0,Cobra Island,21996,mobile,9924,1,1,The Devilfire Empire,5731
52885,110418_1,3763,1,2016-01-02,110418,2016-01-03,desktop,9924,The Devilfire Empire,Glubbdubdrib,53,1,1,2,10,0,Glubbdubdrib,3763,desktop,9924,1,1,The Devilfire Empire,5544
368668,1204242_1,10485,1,2016-01-02,1204242,2016-01-03,desktop,2661,The Devilfire Empire,Kangan,53,1,1,2,5,0,Kangan,5860,desktop,2661,1,1,The Devilfire Empire,5351
531558,1316608_1,41654,1,2016-01-02,1316608,2016-01-03,mobile,4888,Tcherkistan,Pullamawang,53,1,1,2,5,0,Pullamawang,41654,mobile,4888,1,1,Tcherkistan,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244942,5955565_2,54384,1,2017-02-27,5955565,2017-02-28,desktop,8132,Elbonia,Patusan,9,2,1,5,5,1,Elbonia,52327,desktop,8132,1,2,Elbonia,526
787249,6172320_1,6005,1,2017-02-27,6172320,2017-02-28,mobile,9452,The Devilfire Empire,Patusan,9,2,1,10,10,1,Sokovia,34342,mobile,9452,1,2,The Devilfire Empire,367
455641,885453_1,53020,1,2017-02-27,885453,2017-02-28,desktop,9924,Gondal,Gondal,9,2,1,4,4,1,Santa Prisca,44103,desktop,9924,1,2,Gondal,141
175371,890798_1,43524,1,2017-02-27,890798,2017-02-28,desktop,9924,Gondal,Rolisica,9,2,1,4,4,1,Santa Prisca,44103,desktop,9924,1,2,Gondal,514


In [26]:
KEEP_COLS = ['utrip_id', 'city_id', 'label',
             "checkin",
             "hotel_country", ## should be item feature 
             
#               'user_id', 'checkin', 'checkout','device_class', 'affiliate_id',  'booker_country', 
#        'checkin_week', 'checkin_month', 
#               'row_num', 
#              'total_rows',
#               'last',
       'first_hotel_country',
             'first_city_id', 'first_device_class','first_affiliate_id',
#              'first_checkin_quarter',
#              'first_checkin_month',
       'first_booker_country'
            ]
             
df = df[KEEP_COLS]        
df.rename(columns={"checkin":"time",
                  'utrip_id':"user",
                   'city_id':"item"
                  },inplace=True)
print(df.shape)
### opt: drop duplicates, +- time ? 
df.drop_duplicates(subset=['user', 'item', 'hotel_country', 'first_hotel_country',
                           'first_city_id', 'first_booker_country'],inplace=True)
print(df.shape)
print(df.columns.drop(cols_to_order,errors="ignore").tolist())
df

(702158, 10)
(669859, 10)
['user', 'item', 'time', 'hotel_country', 'first_hotel_country', 'first_city_id', 'first_device_class', 'first_affiliate_id', 'first_booker_country']


,user,item,label,time,hotel_country,first_hotel_country,first_city_id,first_device_class,first_affiliate_id,first_booker_country
519018,727105_1,60403,1,2016-01-01,Cobra Island,Cobra Island,60403,mobile,359,The Devilfire Empire
805583,1032571_1,48483,1,2016-01-02,Cobra Island,Cobra Island,21996,mobile,9924,The Devilfire Empire
52885,110418_1,3763,1,2016-01-02,Glubbdubdrib,Glubbdubdrib,3763,desktop,9924,The Devilfire Empire
368668,1204242_1,10485,1,2016-01-02,Kangan,Kangan,5860,desktop,2661,The Devilfire Empire
531558,1316608_1,41654,1,2016-01-02,Pullamawang,Pullamawang,41654,mobile,4888,Tcherkistan
...,...,...,...,...,...,...,...,...,...,...
244942,5955565_2,54384,1,2017-02-27,Patusan,Elbonia,52327,desktop,8132,Elbonia
787249,6172320_1,6005,1,2017-02-27,Patusan,Sokovia,34342,mobile,9452,The Devilfire Empire
455641,885453_1,53020,1,2017-02-27,Gondal,Santa Prisca,44103,desktop,9924,Gondal
175371,890798_1,43524,1,2017-02-27,Rolisica,Santa Prisca,44103,desktop,9924,Gondal


In [39]:
print("Sparsity: (total positives/interactions, divided by all possible user/item combs in %, )")
# print(100*df.shape[0] / (162904 * 3667))
print(100*df.shape[0] / (df['user'].nunique() * df['item'].nunique()))

Sparsity: (total positives/interactions, divided by all possible user/item combs in %, )
0.11213488411403386
0.11213488411403386


### model

In [27]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_WARNINGS"] = "FALSE"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
reset_state("YouTubeRanking")
# reset_state("YouTubeMatch") 

In [28]:
%%time

# train_data, test_data = split_by_ratio_chrono(df, test_size=0.15)
train_data, test_data = split_by_ratio(df, test_size=0.2)

# specify complete columns information
sparse_col = ['first_hotel_country', 
#               'first_city_id','first_device_class','first_affiliate_id', 
              'first_booker_country',
              'first_city_id', 'first_device_class','first_affiliate_id',
             "hotel_country"]
# dense_col = [] #['total_rows',
#              'first_checkin_quarter', 
              #'first_checkin_month',]
user_col = ['first_hotel_country','first_booker_country','first_city_id', 'first_device_class','first_affiliate_id']
item_col = ["hotel_country"]

# train_data, test_data = split_by_ratio(df[user_col+item_col+["label"]+sparse_col+dense_col], test_size=0.15)

# train_data, data_info = DatasetFeat.build_trainset(
#     train_data, user_col, item_col, sparse_col, dense_col
# )
train_data, data_info = DatasetFeat.build_trainset(
    train_data, item_col=item_col, sparse_col=sparse_col,user_col=user_col,
)
print("trainset made")
test_data = DatasetFeat.build_testset(test_data)
print("testset made")

# sample negative items for each record
train_data.build_negative_samples(data_info,num_neg=5)
print("train negs done")
test_data.build_negative_samples(data_info,num_neg=1)
print("test negs done")
print(data_info)  # n_users: 9058, n_items: 456, data sparsity: 0.7836 %
## (min 25 cities): n_users: 162904, n_items: 3667, data sparsity: 0.0977 %

# reset_state("YouTubeRanking")
# reset_state("YouTubeMatch") 
### The YouTubeMatch model assumes no item features.  

## YouTubeRanking  ## YouTubeMatch # DeepFM
ytb_ranking = YouTubeRanking(task="ranking", data_info=data_info,
                             embed_size=16, n_epochs=7, lr=2e-4,
                             batch_size=64, use_bn=True,
                             hidden_units="128,64") ## hidden_units="128,64,32"

ytb_ranking.fit(train_data, verbose=1, shuffle=True,
                eval_data=test_data,
                metrics=[ "loss", "roc_auc", "precision", "recall"]) # "roc_auc",


print(ytb_ranking.evaluate(test_data,metrics=[ "loss", "roc_auc", "precision", "recall"],k=4))
print("recall @90",ytb_ranking.evaluate(test_data,metrics=[ "recall"],k=90))

# # predict preference of user 1 to item 2333
# print("prediction: ", ytb_ranking.predict(user=1, item=33))
# # recommend 7 items for user 1
# print("recommendation: ", ytb_ranking.recommend_user(user=1, n_rec=3))


 ============================== YouTubeRanking ==============================
trainset made
testset made
random neg item sampling elapsed: 1.895s
train negs done
random neg item sampling elapsed: 0.080s
test negs done
n_users: 162904, n_items: 3667, data sparsity: 0.0977 %
Training start time: 2021-01-06 00:43:33


train: 100%|█████████████████████████████████████████████████████████████████████| 72965/72965 [18:15<00:00, 66.58it/s]


Epoch 1 elapsed: 1097.925s


train: 100%|█████████████████████████████████████████████████████████████████████| 72965/72965 [18:15<00:00, 66.59it/s]


Epoch 2 elapsed: 1097.608s


...

train: 100%|█████████████████████████████████████████████████████████████████████| 72965/72965 [18:15<00:00, 66.63it/s]


Epoch 7 elapsed: 1096.930s


eval_rec: 100%|███████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 139.75it/s]


Wall time: 2h 15min 44s


{'loss': 0.806724466471789, 'roc_auc': 0.9827009283558722, 'precision': 0.0885, 'recall': 0.3468333333333333}

### temporal split, min 25:   (results the same on random split ) 
train_data.build_negative_samples(data_info,num_neg=7)
test_data.build_negative_samples(data_info,num_neg=1)

train_data, test_data = split_by_ratio_chrono(df, test_size=0.15)
ytb_ranking = YouTubeRanking(task="ranking", data_info=data_info,
                             embed_size=16, n_epochs=7, lr=2e-4,
                             batch_size=64, use_bn=True,
                             hidden_units="128,64")
============================== YouTubeRanking ==============================
trainset made
testset made
random neg item sampling elapsed: 1.895s
train negs done
random neg item sampling elapsed: 0.080s
test negs done
n_users: 162904, n_items: 3667, data sparsity: 0.0977 %
Training start time: 2021-01-06 00:43:33
train: 100%| 
Epoch 7 elapsed: 1096.930s
Wall time: 2h 15min 44s

Eval (temporal split): 
* {'loss': 0.806, 'roc_auc': 0.982, 'precision': 0.0885, 'recall': 0.3468}
* {recall @ 90 : 'recall': 0.771}


On a random split with same model (might include train on train )  - results are same as with temporal split ; 
* {'loss': 0.806, 'roc_auc': 0.982, 'precision': 0.0885, 'recall': 0.3468}
* recall @90 {'recall': 0.77}

In [29]:
# reset_state("YouTubeRanking")
# data = pd.read_csv("./sample_movielens_merged.csv", sep=",", header=0)
# data["label"] = 1  # convert to implicit data and do negative sampling afterwards

# # split into train and test data based on time
# train_data, test_data = split_by_ratio_chrono(data, test_size=0.2)

# # specify complete columns information
# sparse_col = ["sex", "occupation", "genre1", "genre2", "genre3"]
# dense_col = ["age"]
# user_col = ["sex", "age", "occupation"]
# item_col = ["genre1", "genre2", "genre3"]

# train_data, data_info = DatasetFeat.build_trainset(
#     train_data, user_col, item_col, sparse_col, dense_col
# )
# test_data = DatasetFeat.build_testset(test_data)
# train_data.build_negative_samples(data_info)  # sample negative items for each record
# test_data.build_negative_samples(data_info)
# print(data_info)  # n_users: 5962, n_items: 3226, data sparsity: 0.4185 %

# ytb_ranking = YouTubeRanking(task="ranking", data_info=data_info, embed_size=16, 
#                              n_epochs=3, lr=1e-4, batch_size=512, use_bn=True, 
#                              hidden_units="128,64,32")
# ytb_ranking.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
#                 metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"])

# # predict preference of user 1 to item 2333
# print("prediction: ", ytb_ranking.predict(user=1, item=2333))  
# # recommend 7 items for user 1
# print("recommendation(id, probability): ", ytb_ranking.recommend_user(user=1, n_rec=7))